In [25]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import cv2
import tensorflow as tf


In [2]:
baseDir=os.getcwd()
dataDir=os.path.join(baseDir, 'data')
training_file=os.path.join(dataDir,'train.p')
testing_file=os.path.join(dataDir,'test.p')

In [3]:
with open(training_file,'rb') as f:
    train=pickle.load(f)
    
with open(testing_file,'rb') as f:
    test=pickle.load(f)

In [4]:
train.keys()

dict_keys(['sizes', 'coords', 'features', 'labels'])

In [5]:
test.keys()

dict_keys(['sizes', 'coords', 'features', 'labels'])

In [6]:
train['labels']

array([ 0,  0,  0, ..., 42, 42, 42], dtype=uint8)

In [7]:
X_train,y_train=train['features'],train['labels']
X_test,y_test=test['features'],test['labels']

In [8]:
X_train[0].shape

(32, 32, 3)

In [9]:
print('Image Shape:    {}'.format(X_train[0].shape))
print('Training Set:   {} samples'.format(len(X_train)))
print('Test Set:       {} samples'.format(len(X_test)))
##print('Labels in Set:  {}'.format(np.unique(y_train)))
print('Number of unique classes: {}'.format(len(np.unique(y_train))))

Image Shape:    (32, 32, 3)
Training Set:   39209 samples
Test Set:       12630 samples
Number of unique classes: 43


In [10]:
signnames=pd.read_csv('signnames.csv')
print(signnames)



    ClassId                                           SignName
0         0                               Speed limit (20km/h)
1         1                               Speed limit (30km/h)
2         2                               Speed limit (50km/h)
3         3                               Speed limit (60km/h)
4         4                               Speed limit (70km/h)
5         5                               Speed limit (80km/h)
6         6                        End of speed limit (80km/h)
7         7                              Speed limit (100km/h)
8         8                              Speed limit (120km/h)
9         9                                         No passing
10       10       No passing for vehicles over 3.5 metric tons
11       11              Right-of-way at the next intersection
12       12                                      Priority road
13       13                                              Yield
14       14                                            

In [13]:
from sklearn.utils import shuffle
X_train, y_train=shuffle(X_train, y_train)

In [15]:
from sklearn.model_selection import train_test_split
test_features, valid_features, test_labels, valid_labels= train_test_split(X_test,y_test, test_size=0.60, random_state=42)

In [16]:
print('Number of validation examples = {}'.format(len(valid_labels)))
print('Number of testing examples = {}'.format(len(test_labels)))

Number of validation examples = 7578
Number of testing examples = 5052


In [31]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
epochs = 25
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(valid_features, valid_labels))

Epoch 1/25


ValueError: in user code:

    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\jonat\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C